In [7]:
import pandas as pd
from coordenadas import obter_coordenadas, uf
import sqlite3
from sqlalchemy import create_engine


df_municipios = pd.read_csv('parahybaCities.csv', engine = 'python')

df_municipios = df_municipios.drop(columns=['Tecnologia', 'Tipo de compromisso', 'Compromisso', 'Origem compromisso', 'Operadora com compromisso', 'Status compromisso',
                               'Prazo compromisso', 'Operadora'], axis= 1)

df_municipios = df_municipios.drop_duplicates()

df_municipios['Latitude'] = ''
df_municipios['Longitude'] = ''


for i, municipio in enumerate (df_municipios['Município']):
    lat, lon = obter_coordenadas(municipio, uf)
    df_municipios.at[i, 'Latitude'] = lat
    df_municipios.at[i, 'Longitude'] = lon

print(df_municipios['% moradores cobertos'].unique())

df_municipios['% moradores cobertos'] = pd.to_numeric(df_municipios['% moradores cobertos'], errors='coerce')
df_municipios['% moradores cobertos'].fillna(0, inplace=True)
df_municipios['Longitude'] = pd.to_numeric(df_municipios['Longitude'], errors='coerce')
df_municipios['Latitude'] = pd.to_numeric(df_municipios['Latitude'], errors='coerce')


engine = create_engine('sqlite:///municipios_PB.db')
conn = engine.connect()

df_municipios.to_sql('municipios', conn, if_exists='replace', index=False)
conn.close()



df_municipios = df_municipios.dropna(how='any')

display(df_municipios)

['63,36' '64,71' '93,55' '99,23' '99,57' '68,08' '77,50' '99,95' '68,29'
 '91,01' '82,12' '88,22' '91,95' '87,72' '92,39' '77,40' '99,99' '71,31'
 '99,39' '94,74' '93,17' '96,53' '46,48' '75,61' '55,22' '100,00' '84,38'
 '97,25' '98,35' '77,45' '96,50' '82,29' '87,89' '87,33' '88,11' '91,08'
 '99,69' '63,75' '91,06' '93,76' '86,28' '52,84' '99,19' '86,24' '74,44'
 '63,10' '98,97' '64,72' '96,49' '88,70' '72,15' '86,14' '94,15' '73,66'
 '89,74' '82,62' '85,16' '78,87' '99,41' '99,79' '77,38' '99,03' '81,35'
 '86,90' '91,32' '97,98' '97,71' '75,69' '88,26' '96,48' '88,02' '97,52'
 '84,54' '83,06' '73,72' '98,95' '95,20' '86,78' '91,52' '63,11' '98,20'
 '99,02' '93,92' '97,26' '79,94' '83,38' '89,46' '93,47' '78,07' '88,87'
 '99,75' '72,11' '92,49' '99,76' '94,85' '79,98' '97,48' '87,57' '96,35'
 '98,61' '74,85' '99,46' '96,13' '99,49' '99,81' '89,48' '99,40' '94,28'
 '93,38' '89,59' '74,15' '71,80' '73,86' '79,26' '99,55' '97,03' '81,69'
 '64,88' '83,74' '99,21' '89,87' '97,06' '97,74' '

C:\Users\Joao\AppData\Local\Temp\ipykernel_3420\905991279.py:26: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





,Código IBGE,Município,UF,% área coberta,% moradores cobertos,% domicílios cobertos,Área km2,Moradores,Domicílios,Latitude,Longitude
5,2500304.0,Alagoa Grande,PB,"67,75",0.0,"94,05","320,5623905",28460.0,8012.0,-7.741342,-36.065357
7,2500403.0,Alagoa Nova,PB,"97,82",0.0,"99,29","122,2553871",19678.0,5434.0,-7.430425,-34.910610
10,2500502.0,Alagoinha,PB,"98,21",0.0,"99,54","96,97979893",13533.0,3686.0,-6.785960,-38.083135
14,2500536.0,Alcantil,PB,"54,17",0.0,"68,32","305,393125",5237.0,1565.0,-6.968024,-35.699478
18,2500577.0,Algodão de Jandaíra,PB,"50,86",0.0,"77,99","220,2481783",2359.0,673.0,-7.073396,-36.727400
22,2500601.0,Alhandra,PB,"96,42",0.0,"99,95","182,6612162",17924.0,5017.0,-7.523086,-35.998852
24,2500700.0,São João do Rio do Peixe,PB,"56,32",0.0,"68,33","474,4307206",18144.0,5367.0,-7.752809,-36.317100
26,2500734.0,Amparo,PB,"84,53",0.0,"91,59","121,9838433",2088.0,636.0,-6.696759,-35.536963
29,2500775.0,Aparecida,PB,"57,70",0.0,"81,42","295,7055688",7668.0,2207.0,-7.420246,-38.215678
33,2500809.0,Araçagi,PB,"75,17",0.0,"88,23","231,1550912",17190.0,5004.0,-7.314333,-38.514668


In [8]:
import plotly.express as px


grafico = px.density_mapbox(df_municipios, lon="Longitude", lat="Latitude", z="% moradores cobertos", mapbox_style="open-street-map")

grafico.show()